# Sequence packer

Seuqnce packing for more efficiency.

## Modernbert Source code

From https://github.dev/AnswerDotAI/ModernBERT/blob/main/src/sequence_packer.py

In [ ]:
# Copyright 2024 onwards Answer.AI, LightOn, and contributors
# License: Apache-2.0

import threading
import time
from abc import ABC, abstractmethod
from collections import deque
from typing import Generic, Iterable, NamedTuple, Optional, TypeVar, Any, Union, Sequence

import numpy as np
import torch
from numba import njit

import math, random

# from composer.core.types import Batch
# from composer.core import Time

class BatchSizeWarmupScheduler:
    def __init__(
        self,
        min_batch_size: int,
        max_batch_size: int,
        warmup_tokens,
        # warmup_tokens: Union[str, Time, int],
        world_size: int,
    ):
        self.min_batch_size = min_batch_size
        self.max_batch_size = max_batch_size

        if isinstance(warmup_tokens, str):
            # self.warmup_tokens = Time.from_timestring(warmup_tokens).value
            pass
        elif isinstance(warmup_tokens, Time):
            self.warmup_tokens = warmup_tokens.value
        else:
            self.warmup_tokens = warmup_tokens
        self.warmup_tokens = math.ceil(self.warmup_tokens / world_size)
        self._step_thresholds = self._calculate_step_thresholds()

    def _calculate_step_thresholds(self):
        total_batch_sizes = sum(range(self.min_batch_size, self.max_batch_size))
        steps_per_unit = self.warmup_tokens / total_batch_sizes

        thresholds = []
        cumsum = 0
        for batch_size in range(self.min_batch_size, self.max_batch_size):
            cumsum += batch_size
            steps = math.ceil(steps_per_unit * cumsum)
            thresholds.append(steps)
        return thresholds

    def __call__(self, current_step: int) -> int:
        if current_step >= self.warmup_tokens:
            return self.max_batch_size

        for i, threshold in enumerate(self._step_thresholds):
            if current_step < threshold:
                return self.min_batch_size + i

        # should never hit this, but just in case
        return self.max_batch_size


class SequencePackerBatchOutputTuple(NamedTuple):
    masked_pseqs: torch.Tensor
    labels: Optional[torch.Tensor]
    cu_seq_lens: list[torch.Tensor]
    max_cu_seq_len: list[torch.Tensor]


class SequencePacker(ABC):
    def __init__(
        self,
        # params defining the incoming batches of seqs
        src_iterable: Iterable[list[list[int]]],
        src_batch_size: int,
        src_max_seq_len: int,
        # params defining outgoing batches of pseqs
        out_batch_size: int,
        out_pseq_len: int,
        # params defining internal behavior
        buffer_size: int,
        pad_token_id: int = -1,
        mask_token_id: int = 0,
        ignore_token_id: int = -100,
        mask_prob: float = 0.3,
        seed=42,
        suppress_masking: bool = False,
        batch_size_warmup_min_size: Optional[int] = None,
        batch_size_warmup_tokens = None,
        # batch_size_warmup_tokens: Optional[Union[str, Time]] = None,
        world_size: int = 1,
    ):
        """
        Takes batches of unpacked, unpadded sequences (seqs) to batches of packed and padded sequences (pseqs).

        Every input batch must be a list[list[int]], a list of variable-length sequences of tokens.

        Every output batch is a tuple (masked_inputs:Tensor, labels:Tensor, seq_starts_and_end:list).

        It performs this streamwise, taking an iterable as the source of incoming batches, and
        presents itself as an iterable of outgoing batches.

        Args:
            src_iterable: An iterable (e.g., a DataLoader), whose iterator yields one incoming batch,
                        where a batch is a list of unpadded, variable-length Sequences of token
                        IDs. Since this only needs to be an Iterable, it could also be a generator object
                         like the result of `itertools.batched(dataset_list,batch_size))`

            src_batch_size:  This is the INCOMING batch size, the number of seqs in one batch yielded
                          from `src_iterable`'s iterator.

            src_max_seq_len: The maximum number of tokens in a seq within an incoming batch.

            out_batch_size: the number of pseqs (packed seqs) in one outgoing batch

            out_pseq_len: the number of tokens per packed seq, in every outgoing batch

            buffer_size: The maximum number of seqs which may be buffered internally.

            pad_token_id: The token ID used for padding the space which cannot be filled to reach out_pseq_len.

            mask_token_id: The token ID used for masking tokens in the input sequence.

            ignore_token_id: The token ID used to ignore tokens. Expected to be applied to every non-masked token, so the model only trains on predictions of masked tokens.

            suppress_masking: If True, the sequence packer will not perform masked language modeling.

            batch_size_warmup_min_size: If not None, the sequence packer will gradually increase the batch size from batch_size_warmup_min_size to out_batch_size over the course of the warmup_tokens.
                                    batch_size_warmup_min_size must be a multiple of micro_batch_size.

            batch_size_warmup_tokens: If not None, the sequence packer will gradually increase the batch size from batch_size_warmup_min_size to out_batch_size over the course of the warmup_tokens.

            world_size: The number of processes participating in this training run. batch_size_warmup_min_size is divided by this number.
        """
        assert buffer_size >= out_batch_size, f"required that {buffer_size=} >= {out_batch_size=}"
        self.src_dataloader_len = len(src_iterable)
        self.src_iterable = src_iterable
        self.src_batch_size = src_batch_size
        self.out_batch_size = out_batch_size
        self.out_pseq_len = out_pseq_len
        self.buffer_size = buffer_size
        self.pad_token_id = pad_token_id
        self.mask_token_id = mask_token_id
        self.ignore_token_id = ignore_token_id
        self.mask_prob = mask_prob
        self.suppress_masking = suppress_masking
        # internals
        self.buffer = deque()  # internal buffer holds individual seqs, as tensors.
        # for stats to report packing efficiency.
        self._seqs_consumed = 0
        self._seqs_emitted = 0
        # Set random seed
        self.seed = seed
        self.epoch = -1
        self._token_count = 0
        self.batch_size_scheduler = None
        if batch_size_warmup_min_size is not None and batch_size_warmup_tokens is not None:
            self.batch_size_scheduler = BatchSizeWarmupScheduler(
                batch_size_warmup_min_size, out_batch_size, batch_size_warmup_tokens, world_size
            )
        else:
            self.batch_size_scheduler = None

    @property
    def seqs_emitted(self):
        "Number of seqs, incoming from src_iterable, which have been emitted in OUTGOING batches."
        return self._seqs_emitted

    @property
    def seqs_consumed(self):
        "Number of seqs, incoming from src_iterable, which have been consumed."
        return self._seqs_consumed

    def _reset_state(self):
        self.epoch += 1
        self.buffer.clear()
        self._seqs_consumed = 0
        self._seqs_emitted = 0
        self.np_rng = np.random.default_rng(self.epoch + self.seed)

        # Update the epoch for the sampler
        if isinstance(self.src_iterable, torch.utils.data.dataloader.DataLoader):
            if isinstance(self.src_iterable.sampler, torch.utils.data.distributed.DistributedSampler):
                self.src_iterable.sampler.set_epoch(self.epoch)

    def __iter__(self):
        self._reset_state()
        self.src_iterator = iter(self.src_iterable)
        return self._generate_batches()

    def __len__(self):
        # rather than estimate the packed length of the dataset, we rely on Composer's ability
        # to schedule training the using the number of batches or tokens instead of epochs.
        return None

    def _fill_buffer(self, max_items_to_add=float("inf")) -> int:
        """
        Refills the internal buffer.

        - max_items_to_add: an amount less than or equal to the number of items to add

        Returns: the number of items actually added.

        The default implementation of this simply extends to src.buffer, which is
        initialized as a list in __init__. Subclasses which want to use a different data
        structure for internal buffering should override this method and also add
        code in __init__ to initialize src.buffer appropriately.

        Any implementation of this MUST never place more than self.buffer_size items
        in the internal buffer.
        """
        items_added = 0
        # NOTE: this should be >=, kept as is to match model training code
        # TODO: change if training a new model
        while (self.buffer_size - len(self.buffer)) > self.src_batch_size:
            try:
                # if pulling another batch would fetch more than the requested max, stop
                if max_items_to_add < float("inf"):
                    if (items_added + self.src_batch_size) > max_items_to_add:
                        # print("Not adding, because of max_items_to_fetch")
                        break
                incoming_batch = next(self.src_iterator)
                assert (
                    len(incoming_batch) <= self.src_batch_size
                ), f"expected {len(incoming_batch)=} <= {self.src_batch_size=}"
                for item in incoming_batch:
                    if len(item["input_ids"]) > 0:  # ignore empty sequences
                        self.buffer.append(item["input_ids"])
                        items_added += 1
                        self._seqs_consumed += 1
            except StopIteration:
                break
        return items_added

    def _generate_batches(self):
        """
        Generates batches of packed sequences.

        The returned generator's iterator will always, when next() is called on it, either:
         - return a valid tuple batch (masked_batch, labels, cu_seq_lens,max_seq_lens)
         - raise StopIteration
        """
        while True:
            retval = self._create_batch()
            if retval is None:
                break
            batch, lst_cu_seq_lens = retval

            assert isinstance(retval, tuple), f"Unexpected {type(retval)=}"
            assert isinstance(retval[0], np.ndarray), f"Unexpected {type(retval[0])=}"
            assert isinstance(retval[1], list), f"Unexpected {type(retval[1])=}"

            cu_seq_lens = [torch.tensor(x, dtype=torch.int32) for x in lst_cu_seq_lens]
            max_seq_lens = [torch.max(x[1:] - x[:-1]).item() for x in cu_seq_lens]
            assert isinstance(cu_seq_lens, list), f"Unexpected {type(cu_seq_lens)=}"
            if self.suppress_masking:
                yieldval = {
                    "input_ids": torch.from_numpy(batch),
                    "labels": None,
                    "cu_seqlens": cu_seq_lens,
                    "max_seqlen": max_seq_lens,
                }
            else:
                (masked_batch, labels) = SequencePacker.mlm_masking(
                    batch, self.mask_prob, self.mask_token_id, self.pad_token_id, self.ignore_token_id, self.np_rng
                )
                yieldval = {
                    "input_ids": torch.from_numpy(masked_batch),
                    "labels": torch.from_numpy(labels),
                    "cu_seqlens": cu_seq_lens,
                    "max_seqlen": max_seq_lens,
                    "attention_mask": torch.from_numpy(np.where(batch == self.pad_token_id, 0, 1)),
                }
                self._token_count += yieldval["attention_mask"].sum().item()
            # # assert isinstance(yieldval[0], torch.Tensor), f"Unexpected {type(yieldval[0])=}"
            # if not self.suppress_masking:
            #     assert isinstance(yieldval[1], torch.Tensor), f"Unexpected {type(yieldval[1])=}"
            # assert isinstance(yieldval[2], list), f"Unexpected {type(yieldval[2])=}"
            # if yieldval[2]:
            #     assert isinstance(yieldval[2][0], torch.Tensor), f"Unexpected {type(yieldval[2][0])=}"
            yield yieldval

    @staticmethod
    def mlm_masking(
        seq: np.ndarray,
        mask_prob: float,
        mask_token: int,
        pad_token: int = -1,
        ignore_index: int = -100,
        np_rng=np.random.default_rng(),
    ) -> tuple[np.ndarray, np.ndarray]:
        """
        Prepare masked tokens inputs/labels for masked language modeling: 80% MASK, 10% random, 10% original.

        This is exactly a numpy version of transformers' `DataCollatorForLanguageModeling.torch_mask_tokens`
        https://github.com/huggingface/transformers/blob/main/src/transformers/data/data_collator.py#L827

        It performs masking in a way that produces on expectation the following masked inputs:
         - (1-mask_prob) of the original positions will be untouched.
         - mask_prob * 80%  of the original positions get replaced with a mask token
         - mask_prob * 10%  of the original positions get replaced with a random token
         - mask_prob * 10%  of the original positions also remain untouched.
        This generates the masked_inputs.

        It also generates a labels array, which has ignore tokens in the (1-mask_prob) positions

        These proportions are expectation values since the random transformation is performed
        independently per element. (This is why it is agnostic wrt shape.)

        Args:
          seq (np.ndarray): the input token IDs (e.g., a sequence, or batch of seqs)
          mask_prob (float): probability of initially masking a token, in the first "wave" of masking
          mask_token (int): token to use for masking
          ignore_index (int): the token indicating that position should be ignored during training. We call it `ignore_index` to conform to the API of the cross entropy loss function.

        Returns:
            tuple[np.array,np.array]: (masked_seq, labels)
                masked_seq: the input seq with some tokens replaced by `mask_token`
                labels: the original input seq with non-masked tokens replaced by `ignore_index`
        """
        # Create labels
        labels = np.where(seq == pad_token, ignore_index, seq)

        # Create a single mask
        rand = np_rng.random(seq.shape)

        # Partition the probability space appropriately using a single mask
        # 80% of the time, we mask the token
        mask_mask = rand < mask_prob * 0.8
        # 10% of the time, we replace the token with a random token
        random_mask = (rand >= mask_prob * 0.8) & (rand < mask_prob * 0.9)
        # 10% of the time, we keep the token the same
        keep_mask = (rand >= mask_prob * 0.9) & (rand < mask_prob)

        # We only compute loss over the tokens marked for masking
        labels = np.where(mask_mask | random_mask | keep_mask, labels, ignore_index)

        # Apply masking
        seq = np.where(mask_mask, mask_token, seq)

        # Apply random replacement
        random_words = np_rng.integers(0, np.max(seq) + 1, size=seq.shape)
        seq = np.where(random_mask, random_words, seq)

        return seq, labels

    @abstractmethod
    def _create_batch(self) -> Optional[tuple[np.ndarray, list[list[int]]]]:
        """
        Returns a batch of packed sequences with its cumulative seq length information.

        Or else, returns None if it cannot build a full outgoing batch.

        Must mutate self.buffer to remove the sequences that are packed into the batch.

        Returns:
            (out_batch,cumulative_seq_len):tuple[torch.tensor, list[list[int]]]
            where:
                - out_batch is a tensor of shape (out_batch_size, out_pseq_len);
                - cum_seq_lens is a list of lists, where the outer list is of len out_batch_size,
                    and each inner list is of varying length, and contains the start positions of
                    every seq in the pseq, and the end position of the last seq in the pseq. This end
                    position is necessary to communicate if any padding tokens were added.
        """
        pass


@njit
def find_best_fit(remaining_spaces, seq_len):
    valid_spaces = seq_len <= remaining_spaces
    if np.any(valid_spaces):
        valid_space_sizes = remaining_spaces[valid_spaces]
        best_fit_idx = np.argmin(valid_space_sizes)
        return np.arange(len(remaining_spaces))[valid_spaces][best_fit_idx]
    return -1


class GreedyBestFitSequencePacker(SequencePacker):
    @classmethod
    def from_composer(
        cls,
        src_iterable: Iterable[list[list[int]]],
        batch_size: int = 512,
        micro_batch_size: int = 32,
        max_seq_len: int = 1024,
        buffer_size: int = 5120,
        # token values
        pad_token_id: int = -1,
        mask_token_id: int = 0,
        ignore_token_id: int = -100,
        mask_prob: float = 0.3,
        # transform values
        seed=42,
        suppress_masking=False,
        batch_size_warmup_min_size: Optional[int] = None,
        batch_size_warmup_tokens = None,
        # batch_size_warmup_tokens: Optional[Union[str, Time]] = None,
        world_size: int = 1,
    ) -> "GreedyBestFitSequencePacker":
        if batch_size_warmup_min_size is not None:
            if batch_size_warmup_min_size % micro_batch_size != 0:
                raise ValueError(f"{batch_size_warmup_min_size=} must be a multiple of {micro_batch_size=}")
            batch_size_warmup_min_size = int(batch_size_warmup_min_size / micro_batch_size)
        return cls(
            # input shape
            src_iterable=src_iterable,
            src_batch_size=batch_size,
            src_max_seq_len=max_seq_len,
            # output shape
            out_batch_size=int(batch_size / micro_batch_size),
            out_pseq_len=int(micro_batch_size * max_seq_len),
            # internal
            buffer_size=buffer_size,
            # transformation
            pad_token_id=pad_token_id,
            mask_token_id=mask_token_id,
            ignore_token_id=ignore_token_id,
            mask_prob=mask_prob,
            seed=seed,
            suppress_masking=suppress_masking,
            batch_size_warmup_min_size=batch_size_warmup_min_size,
            batch_size_warmup_tokens=batch_size_warmup_tokens,
            world_size=world_size,
        )

    def _create_batch(self) -> Optional[tuple[np.ndarray, list[list[int]]]]:
        if self.batch_size_scheduler:
            self.out_batch_size = self.batch_size_scheduler(self._token_count)

        batch = np.full(
            (self.out_batch_size, self.out_pseq_len), self.pad_token_id, dtype=np.int64
        )  # the pseqs being constructed
        seq_counts = np.zeros(self.out_batch_size, dtype=np.int32)  # the count of seqs per pseq
        cum_seq_lens = [[0] for _ in range(self.out_batch_size)]
        remaining_spaces = np.full(
            (self.out_batch_size,), self.out_pseq_len, dtype=np.int32
        )  # the space remaining per pseq
        temp_buffer = []

        while True:
            # Check if buffer has more items, and if not replenish
            if not self.buffer:
                items_to_fetch = self.buffer_size - len(temp_buffer)
                items_added = self._fill_buffer(items_to_fetch)
                if items_added == 0:
                    break

            seq = self.buffer.popleft()
            seq_len = len(seq)

            # Find the best fit (smallest space that can accommodate the sequence)
            best_fit_idx = find_best_fit(remaining_spaces, seq_len)
            if best_fit_idx != -1:
                end_pos = self.out_pseq_len - remaining_spaces[best_fit_idx]
                batch[best_fit_idx, end_pos : end_pos + seq_len] = seq
                seq_counts[best_fit_idx] += 1
                remaining_spaces[best_fit_idx] -= seq_len
                cum_seq_lens[best_fit_idx].append(cum_seq_lens[best_fit_idx][-1] + seq_len)
            else:
                # Can't fit the sequence, save for next batch
                temp_buffer.append(seq)

        # Add any sequences we skipped back to the start of the buffer
        self.buffer.extendleft(temp_buffer)

        if np.all(seq_counts > 0):
            self._seqs_emitted += np.sum(seq_counts)
            for x in cum_seq_lens:
                if x[-1] != self.out_pseq_len:
                    x.append(self.out_pseq_len)
            return batch, cum_seq_lens
        else:
            # If we can't form a full batch, we return None to signal the end
            return None


T = TypeVar("T")


class BufferedIterable(Generic[T]):
    def __init__(self, iterable: Iterable[T], buffer_size: int):
        """
        Args:
          - iterable: an object which generates a fresh iterator on iter() and which implements len()
        """
        self.iterable = iterable
        self.buffer_size = buffer_size

    def __iter__(self):
        return BufferedIterator(self.iterable, self.buffer_size)


class BufferedIterator(Generic[T]):
    def __init__(self, iterable: Iterable[T], buffer_size: int):
        self.iterator = iter(iterable)
        self.buffer = deque(maxlen=buffer_size)
        self.buffer_size = buffer_size
        self.lock = threading.Lock()
        self.exhausted = False
        self.filler_thread = threading.Thread(target=self._background_fill, daemon=True)
        self.filler_thread.start()

    def _background_fill(self):
        # Fill up the buffer, whenever possible, in the background
        while not self.exhausted:
            if len(self.buffer) < self.buffer_size:
                try:
                    item = next(self.iterator)
                    with self.lock:
                        self.buffer.append(item)
                except StopIteration:
                    self.exhausted = True
                    break
            else:
                time.sleep(0.01)  # Sleep for a bit to avoid busy waiting

    def __iter__(self):
        return self

    def __next__(self) -> T:
        while True:
            if not self.buffer:
                if self.exhausted:
                    # We've exhausted the iterator and the buffer so we're done
                    raise StopIteration
                else:
                    # The buffer is empty but the iterator is not exhausted yet.
                    # Let's give the filler thread a chance to add items to the buffer
                    time.sleep(0.01)
            else:
                with self.lock:
                    return self.buffer.popleft()


def split_packed_batch(batch: Any, microbatch_size: Union[int, float], padding_tolerance=1.0) -> Sequence:
    # NOTE: Packed sequences are already packed into a microbatch size worth of tokens.
    # So to correctly return a microbatch worth of data, we will simply return each item (i.e. microbatch_size 1)

    num_items = batch["input_ids"].shape[0]
    split_inputs = [x.squeeze() for x in batch["input_ids"].split(1)]
    split_labels = [x.squeeze() for x in batch["labels"].split(1)]
    split_attention_masks = [x.squeeze() for x in batch["attention_mask"].split(1)]
    split_cu_seqlens = batch["cu_seqlens"]

    result = []
    for i in range(num_items):
        attention_mask = split_attention_masks[i]
        padding_amount = 1 - (attention_mask.sum() / len(attention_mask))

        if padding_amount > padding_tolerance:
            last_non_pad = attention_mask.nonzero().max()
            input_ids = split_inputs[i][: last_non_pad + 1]
            labels = split_labels[i][: last_non_pad + 1]
            cu_seqlens = split_cu_seqlens[i][:-1]
            attention_mask = attention_mask[: last_non_pad + 1]
        else:
            input_ids = split_inputs[i]
            labels = split_labels[i]
            cu_seqlens = split_cu_seqlens[i]

        result.append(
            {
                "input_ids": input_ids,
                "labels": labels,
                "cu_seqlens": cu_seqlens,
                "max_seqlen": batch["max_seqlen"][i],
                "attention_mask": attention_mask,
            }
        )

    assert all([x["input_ids"].shape[-1] == y["cu_seqlens"][-1] for x, y in zip(result, result)])
    return result


def get_num_samples_in_packed_batch(batch) -> int:
    # Number of sequences can be inferred from cu_seqlens arrays
    cu_seqlens = batch["cu_seqlens"]
    if isinstance(cu_seqlens, torch.Tensor):
        return cu_seqlens.size()[0] - 1
    elif isinstance(cu_seqlens, list):
        return sum([x.size()[0] - 1 for x in batch["cu_seqlens"]])
    else:
        raise TypeError('Expected a batch with a "cu_seqlens" key of type list or Tensor')

## Dataset

**AI Response**

I'll show you a simple example of how to use the SequencePacker, specifically the GreedyBestFitSequencePacker implementation. Let's create a minimal working example:

```python
# First, let's create some sample data
import random

# Create a simple dataset of variable length sequences
class SimpleDataset:
    def __init__(self, num_samples=100, min_len=10, max_len=100, vocab_size=1000):
        self.samples = []
        for _ in range(num_samples):
            seq_len = random.randint(min_len, max_len)
            self.samples.append({"input_ids": [random.randint(1, vocab_size-1) for _ in range(seq_len)]})
    
    def __len__(self): return len(self.samples)
    def __getitem__(self, idx): return self.samples[idx]

# Create a batch sampler
from torch.utils.data import DataLoader

dataset = SimpleDataset(num_samples=100)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# Create the sequence packer
packer = GreedyBestFitSequencePacker(
    src_iterable=dataloader,
    src_batch_size=8,
    src_max_seq_len=100,
    out_batch_size=2,
    out_pseq_len=200,
    buffer_size=32,
    pad_token_id=-1,
    mask_token_id=0,
    ignore_token_id=-100,
    mask_prob=0.15,
    suppress_masking=False
)

# Use the packer
for i, batch in enumerate(packer):
    if i >= 3: break  # Just show first few batches
    
    # Unpack the batch
    input_ids = batch["input_ids"]
    labels = batch["labels"]
    cu_seqlens = batch["cu_seqlens"]
    
    print(f"Batch {i}:")
    print(f"  Input shape: {input_ids.shape}")
    print(f"  Labels shape: {labels.shape}")
    print(f"  Cumulative sequence lengths: {[s.tolist() for s in cu_seqlens]}")
    print(f"  Sequences per packed sequence: {[len(s)-1 for s in cu_seqlens]}")
    print()
```

This example:

1. Creates a simple dataset with variable length sequences
2. Sets up a DataLoader to batch the sequences
3. Creates a GreedyBestFitSequencePacker that:
   - Takes batches of 8 sequences from the DataLoader
   - Produces output batches with 2 packed sequences, each with length 200
   - Uses a buffer size of 32 sequences
   - Applies 15% masking for MLM training
4. Iterates through the packed batches and prints information about each one

The key benefit of the sequence packer is efficiently packing variable-length sequences into fixed-length batches, which maximizes GPU utilization for training language models.

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
class SimpleDataset:
    def __init__(self, num_samples=100, min_len=10, max_len=100, vocab_size=1000):
        self.samples = []
        for _ in range(num_samples):
            seq_len = random.randint(min_len, max_len)
            self.samples.append({"input_ids": [random.randint(1, vocab_size-1) for _ in range(seq_len)]})
    
    def __len__(self): return len(self.samples)
    def __getitem__(self, idx): return self.samples[idx]

In [ ]:
ds = SimpleDataset(num_samples=5, min_len=10, max_len=50, vocab_size=50)
ds[0]

{'input_ids': [3,
  24,
  18,
  9,
  21,
  7,
  16,
  47,
  42,
  33,
  41,
  44,
  43,
  4,
  16,
  42,
  5,
  11]}

In [ ]:
len(ds)

5

In [ ]:
[len(ds[i]['input_ids']) for i in range(len(ds))]

[18, 23, 21, 34, 29]

In [ ]:
def simple_collate(batch):
    return batch

dl = DataLoader(ds, batch_size=8, shuffle=True, collate_fn=simple_collate)
batch = next(iter(dl))
print(len(batch))
print([len(item['input_ids']) for item in batch])

5
[18, 34, 23, 21, 29]


In [ ]:
larger_ds = SimpleDataset(num_samples=50, min_len=20, max_len=50, vocab_size=1000)
larger_dl = DataLoader(larger_ds, batch_size=8, shuffle=True, collate_fn=simple_collate)
smaller_packer = GreedyBestFitSequencePacker(
    src_iterable=larger_dl,
    src_batch_size=8,
    src_max_seq_len=50,  # Match our max sequence length
    out_batch_size=2,    # Create 2 packed sequences per batch
    out_pseq_len=100,    # Each packed sequence can hold ~2-5 of our sequences
    buffer_size=32,
    pad_token_id=-1,
    mask_token_id=0,
    ignore_token_id=-100,
    mask_prob=0.15,
    suppress_masking=False
)

for i, batch in enumerate(smaller_packer):
    if i >= 2: break
    
    input_ids = batch["input_ids"]
    labels = batch["labels"]
    cu_seqlens = batch["cu_seqlens"]
    
    print(f"Batch {i}:")
    print(f"  Input shape: {input_ids.shape}")
    print(f"  Labels shape: {labels.shape}")
    print(f"  Cumulative sequence lengths: {[s.tolist() for s in cu_seqlens]}")
    print(f"  Sequences per packed sequence: {[len(s)-1 for s in cu_seqlens]}")
    print()

Batch 0:
  Input shape: torch.Size([2, 100])
  Labels shape: torch.Size([2, 100])
  Cumulative sequence lengths: [[0, 21, 43, 68, 90, 100], [0, 38, 74, 99, 100]]
  Sequences per packed sequence: [5, 4]

Batch 1:
  Input shape: torch.Size([2, 100])
  Labels shape: torch.Size([2, 100])
  Cumulative sequence lengths: [[0, 30, 70, 92, 100], [0, 46, 92, 100]]
  Sequences per packed sequence: [4, 3]



In [ ]:
batch["input_ids"]

tensor([[601, 650, 344, 466,  94, 287,  43,   0,  88, 304, 625,  33, 245, 453,
         753, 186, 158, 154, 121,   0, 444, 378, 197, 924, 330,  24, 466,   0,
         467, 633, 200, 691,   0, 862,   0,   0, 733, 243, 552, 132, 947, 841,
           0, 681, 512,  73,   0,  81, 262,   0, 168, 639, 122, 366, 556, 429,
         986, 800, 170, 152, 466, 351,  42,   0, 150, 792, 696, 671, 784, 794,
         410, 245, 995, 288,  18, 814, 867, 742, 924, 392,   0, 213,   0, 814,
         712, 482, 177, 925, 118, 713, 130, 755, 469,  -1,  -1,  -1,  -1,  -1,
          -1,  -1],
        [340, 325, 653, 336, 724, 876, 771, 572, 199, 529, 931, 658, 691,  29,
         516, 726, 913,   0, 742, 360, 906, 390,   0, 731,   0,   0, 843,   0,
         147, 675, 136,   0,   0, 545,   0, 597, 462, 537,  85, 872,   0,   0,
         119,   0, 371, 407, 302, 357, 607, 180, 281, 784, 969, 582, 883, 304,
         496, 924,   0, 738, 891,  59,   0, 660, 254,   0, 464,   0,   0, 999,
          35, 863, 344, 471,   0

In [ ]:
batch["labels"]

tensor([[-100, -100, -100, -100, -100, -100, -100,   10, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100,  762,   17, -100, -100, -100, -100,
         -100, -100, -100,  908, -100, -100, -100, -100,  991, -100,  199,  661,
         -100, -100, -100, -100, -100, -100,  809, -100, -100, -100,  187, -100,
         -100,  567, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100,  512, -100, -100, -100, -100, -100, -100,  410, -100,
         -100, -100, -100, -100, -100, -100,  489, -100,  725, -100,  653, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100],
        [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100,  821, -100, -100, -100, -100,  165, -100,
          922,  221, -100,  696, -100, -100, -100,  161,   90, -100,  711, -100,
         -100, -100, -100, -100,  535,  784, -100,  101, -100, -100, -100, 

In [ ]:
batch["cu_seqlens"]

[tensor([  0,  23,  67,  93, 100], dtype=torch.int32),
 tensor([  0,  42,  89, 100], dtype=torch.int32)]

**AI Prompt**

What are input_ids, labels, and cu_seqlens?

**AI Response**

Looking at the output from the SequencePacker, let me explain each of these components:

1. **input_ids**: 
   - This is a tensor containing the packed token sequences
   - Shape: [out_batch_size, out_pseq_len] (in this case [2, 100])
   - Contains the actual token IDs, with some tokens masked (replaced with mask_token_id=0)
   - Padding tokens are represented by pad_token_id=-1

2. **labels**:
   - This is a tensor used for the masked language modeling (MLM) training objective
   - Shape: Same as input_ids [2, 100]
   - Contains:
     - The original token IDs at positions that were masked
     - ignore_token_id (-100) at positions that weren't masked
     - The model is trained to predict only the masked tokens during training

3. **cu_seqlens** (cumulative sequence lengths):
   - A list of tensors (one per packed sequence) that tells you where each original sequence begins and ends within the packed sequence
   - Each tensor contains indices that mark the boundaries between sequences
   - For example, [0, 40, 83, 100] means:
     - First sequence: positions 0-39 (40 tokens)
     - Second sequence: positions 40-82 (43 tokens)
     - Third sequence: positions 83-99 (17 tokens)
     - The last value (100) is the total length of the packed sequence

The packer efficiently combines multiple variable-length sequences into fixed-length packed sequences, keeping track of where each original sequence begins and ends. This improves training efficiency by reducing padding waste.

## Autoregressive

**AI Prompt**

I want to use sequence packer for autoregressive model.

**AI Response**

For using the SequencePacker with an autoregressive model, you'll need to modify a few things. The current implementation is designed for masked language modeling (MLM) rather than autoregressive training.

Here's how you can adapt it for autoregressive models:

```python
class AutoregressiveSequencePacker(GreedyBestFitSequencePacker):
    def __init__(self, *args, **kwargs):
        kwargs['suppress_masking'] = True
        super().__init__(*args, **kwargs)
    
    def _generate_batches(self):
        """Override to create shifted inputs and labels for autoregressive training"""
        for batch_dict in super()._generate_batches():
            input_ids = batch_dict["input_ids"]
            
            # For autoregressive models:
            # - inputs are all tokens except the last one
            # - labels are all tokens except the first one
            inputs = input_ids.clone()
            labels = input_ids.clone()
            
            # Create attention mask (1 for tokens, 0 for padding)
            attention_mask = (inputs != self.pad_token_id).long()
            
            # For each packed sequence, create proper labels
            for i, cu_seqlens in enumerate(batch_dict["cu_seqlens"]):
                for j in range(len(cu_seqlens) - 1):
                    start, end = cu_seqlens[j].item(), cu_seqlens[j+1].item()
                    
                    # Set labels to ignore_token_id at sequence boundaries
                    if j < len(cu_seqlens) - 2:  # Not the last sequence
                        labels[i, end-1] = self.ignore_token_id
            
            # Create the causal mask for autoregressive attention
            # Each token can only attend to itself and previous tokens
            seq_len = inputs.size(1)
            causal_mask = torch.tril(torch.ones(seq_len, seq_len)).view(1, 1, seq_len, seq_len)
            
            yield {
                "input_ids": inputs,
                "labels": labels,
                "attention_mask": attention_mask,
                "causal_mask": causal_mask,
                "cu_seqlens": batch_dict["cu_seqlens"],
                "max_seqlen": batch_dict["max_seqlen"]
            }
```

And here's how you'd use it:

```python
autoregressive_packer = AutoregressiveSequencePacker(
    src_iterable=larger_dl,
    src_batch_size=8,
    src_max_seq_len=50,
    out_batch_size=2,
    out_pseq_len=100,
    buffer_size=32,
    pad_token_id=-1,
    ignore_token_id=-100
)

for i, batch in enumerate(autoregressive_packer):
    if i >= 1: break
    
    print(f"Batch {i}:")
    print(f"  Input shape: {batch['input_ids'].shape}")
    print(f"  Labels shape: {batch['labels'].shape}")
    print(f"  Attention mask shape: {batch['attention_mask'].shape}")
    print(f"  Causal mask shape: {batch['causal_mask'].shape}")
    print(f"  Cumulative sequence lengths: {[s.tolist() for s in batch['cu_seqlens']]}")
```

This implementation:
1. Inherits from GreedyBestFitSequencePacker but turns off masking
2. Overrides _generate_batches to create proper inputs and labels for autoregressive training
3. Adds a causal attention mask to ensure tokens only attend to previous tokens
4. Sets labels at sequence boundaries to ignore_token_id to avoid the model learning to predict across sequence boundaries

In [ ]:
class AutoregressiveSequencePacker(GreedyBestFitSequencePacker):
    def __init__(self, *args, **kwargs):
        kwargs['suppress_masking'] = True
        super().__init__(*args, **kwargs)
    
    def _generate_batches(self):
        """Override to create shifted inputs and labels for autoregressive training"""
        for batch_dict in super()._generate_batches():
            input_ids = batch_dict["input_ids"]
            
            # Create the shifted inputs (tokens 0 to n-1) and labels (tokens 1 to n)
            inputs = input_ids.clone()
            # For labels, we shift by 1 (each position predicts the next token)
            labels = torch.roll(input_ids, shifts=-1, dims=1).clone()
            
            # Create attention mask (1 for tokens, 0 for padding)
            attention_mask = (inputs != self.pad_token_id).long()
            
            # For each packed sequence, handle sequence boundaries
            for i, cu_seqlens in enumerate(batch_dict["cu_seqlens"]):
                for j in range(len(cu_seqlens) - 1):
                    start, end = cu_seqlens[j].item(), cu_seqlens[j+1].item()
                    
                    # Set the label at the last position of each sequence to ignore_token_id
                    # We don't want to predict across sequence boundaries
                    labels[i, end-1] = self.ignore_token_id
            
            # Create the causal mask for autoregressive attention
            seq_len = inputs.size(1)
            causal_mask = torch.tril(torch.ones(seq_len, seq_len)).view(1, 1, seq_len, seq_len)
            
            yield {
                "input_ids": inputs,
                "labels": labels,
                "attention_mask": attention_mask,
                "causal_mask": causal_mask,
                "cu_seqlens": batch_dict["cu_seqlens"],
                "max_seqlen": batch_dict["max_seqlen"]
            }

In [ ]:
autoregressive_packer = AutoregressiveSequencePacker(
    src_iterable=larger_dl,
    src_batch_size=8,
    src_max_seq_len=50,
    out_batch_size=2,
    out_pseq_len=100,
    buffer_size=32,
    pad_token_id=-1,
    ignore_token_id=-100
)

for i, batch in enumerate(autoregressive_packer):
    if i >= 1: break
    
    print(f"Batch {i}:")
    print(f"  Input shape: {batch['input_ids'].shape}")
    print(f"  Labels shape: {batch['labels'].shape}")
    print(f"  Attention mask shape: {batch['attention_mask'].shape}")
    print(f"  Causal mask shape: {batch['causal_mask'].shape}")
    print(f"  Cumulative sequence lengths: {[s.tolist() for s in batch['cu_seqlens']]}")

In [ ]:
batch['input_ids']

In [ ]:
batch['labels']

In [ ]:
batch['attention_mask']

In [ ]:
batch['causal_mask']

**AI Prompt**

Does it look good to you?

**AI Response**

Yes, the implementation looks very good! The AutoregressiveSequencePacker is correctly configured for autoregressive training:

1. **Input/Label Shifting**: The labels are properly shifted (using `torch.roll`) so that each position predicts the next token.

2. **Sequence Boundary Handling**: The code correctly identifies the end of each sequence (using `cu_seqlens`) and sets the label at those positions to `ignore_token_id` (-100). This prevents the model from learning to predict across sequence boundaries.

3. **Attention Masking**: Two important masks are created:
   - `attention_mask`: Distinguishes real tokens (1) from padding tokens (0)
   - `causal_mask`: A lower triangular matrix ensuring each token can only attend to itself and previous tokens

4. **Packed Sequences**: The original packing functionality is preserved, efficiently combining multiple sequences into fixed-length batches.

The output shows everything is working correctly:
- The input and label tensors have the expected shapes
- The cumulative sequence lengths show we've packed multiple sequences (4 in the first packed sequence, 4 in the second)
- The labels are properly shifted, with `-100` at sequence boundaries
- The attention mask correctly identifies non-padding tokens
- The causal mask has the expected lower triangular structure

This implementation will work well for training autoregressive models like GPT with packed sequences, which should improve training efficiency by reducing padding waste.

**AI Prompt**

How do I use this now?

**AI Response**

Here's how to use the AutoregressiveSequencePacker with a language model like GPT:

```python
# 1. Create your dataset and dataloader
train_dataset = SimpleDataset(num_samples=10000, min_len=20, max_len=200)
train_dataloader = DataLoader(
    train_dataset, 
    batch_size=16, 
    shuffle=True, 
    collate_fn=simple_collate
)

# 2. Create the sequence packer
packer = AutoregressiveSequencePacker(
    src_iterable=train_dataloader,
    src_batch_size=16,
    src_max_seq_len=200,
    out_batch_size=4,
    out_pseq_len=512,
    buffer_size=64,
    pad_token_id=-1,
    ignore_token_id=-100
)

# 3. Create your model (example with a simple GPT-style model)
from transformers import GPT2Config, GPT2LMHeadModel

model_config = GPT2Config(
    vocab_size=1000,
    n_positions=512,
    n_ctx=512,
    n_embd=768,
    n_layer=6,
    n_head=12
)
model = GPT2LMHeadModel(model_config)

# 4. Training loop
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.train()
for epoch in range(3):
    for batch in packer:
        # Move batch to device
        inputs = batch["input_ids"].to(device)
        labels = batch["labels"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        
        # Forward pass
        outputs = model(
            input_ids=inputs,
            attention_mask=attention_mask,
            labels=labels
        )
        
        loss = outputs.loss
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print(f"Loss: {loss.item()}")
```

For inference with the trained model:

```python
def generate_text(model, prompt, max_length=50):
    # Convert prompt to tensor
    input_ids = torch.tensor([prompt], dtype=torch.long).to(device)
    
    # Generate
    with torch.no_grad():
        output = model.generate(
            input_ids=input_ids,
            max_length=max_length,
            do_sample=True,
            top_p=0.95,
            temperature=0.8
        )
    
    return output[0]

# Example prompt (list of token IDs)
prompt = [10, 25, 36, 42]
generated = generate_text(model, prompt)
print(generated)
```

Key points:
1. The packer efficiently combines sequences for better GPU utilization
2. The model receives properly formatted inputs with causal masking
3. The labels are shifted and have sequence boundaries masked
4. The loss is automatically computed only on valid positions (ignoring padding and sequence boundaries)

In [ ]:
# 1. Create your dataset and dataloader
train_dataset = SimpleDataset(num_samples=10000, min_len=20, max_len=200)
train_dataloader = DataLoader(
    train_dataset, 
    batch_size=16, 
    shuffle=True, 
    collate_fn=simple_collate
)

# 2. Create the sequence packer
packer = AutoregressiveSequencePacker(
    src_iterable=train_dataloader,
    src_batch_size=16,
    src_max_seq_len=200,
    out_batch_size=4,
    out_pseq_len=512,
    buffer_size=64,
    pad_token_id=-1,
    ignore_token_id=-100
)

# 3. Create your model (example with a simple GPT-style model)
from transformers import GPT2Config, GPT2LMHeadModel

model_config = GPT2Config(
    vocab_size=1000,
    n_positions=512,
    n_ctx=512,
    n_embd=768,
    n_layer=6,
    n_head=12
)
model = GPT2LMHeadModel(model_config)

# 4. Training loop
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.train()
for epoch in range(3):
    for batch in packer:
        # Move batch to device
        inputs = batch["input_ids"].to(device)
        labels = batch["labels"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        
        # Forward pass
        outputs = model(
            input_ids=inputs,
            attention_mask=attention_mask,
            labels=labels
        )
        
        loss = outputs.loss
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print(f"Loss: {loss.item()}")


In [ ]:
import pdb; pdb.pm()

> /home/kappa/miniforge3/envs/torch_latest/lib/python3.11/site-packages/torch/nn/functional.py(2551)embedding()
   2549         # remove once script supports set_grad_enabled
   2550         _no_grad_embedding_renorm_(weight, input, max_norm, norm_type)
-> 2551     return torch.embedding(weight, input, padding_idx, scale_grad_by_freq, sparse)
   2552 
   2553 

ipdb> q


In [ ]:
def generate_text(model, prompt, max_length=50):
    # Convert prompt to tensor
    input_ids = torch.tensor([prompt], dtype=torch.long).to(device)
    
    # Generate
    with torch.no_grad():
        output = model.generate(
            input_ids=input_ids,
            max_length=max_length,
            do_sample=True,
            top_p=0.95,
            temperature=0.8
        )
    
    return output[0]

# Example prompt (list of token IDs)
prompt = [10, 25, 36, 42]
generated = generate_text(model, prompt)
print(generated)


In [ ]:
# Print the maximum token ID in your data
max_token = 0
for batch in packer:
    curr_max = batch["input_ids"].max().item()
    if curr_max > max_token:
        max_token = curr_max
    break
print(f"Maximum token ID in data: {max_token}")

Maximum token ID in data: 999


In [ ]:
# Check for negative values besides the padding token
for batch in packer:
    # Find all unique values
    unique_values = torch.unique(batch["input_ids"])
    print(f"Unique values in input_ids: {unique_values}")
    
    # Check if there are negative values other than pad_token_id
    neg_values = unique_values[unique_values < 0]
    print(f"Negative values: {neg_values}")
    
    # Count occurrences of each value
    for val in neg_values:
        count = (batch["input_ids"] == val).sum().item()
        print(f"Value {val} occurs {count} times")
    
    break


Unique values in input_ids: tensor([ -1,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,
         43,  45,  46,  47,  49,  50,  52,  53,  55,  56,  57,  58,  59,  60,
         61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  74,  75,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
         92,  93,  94,  95,  96,  98,  99, 100, 101, 102, 104, 105, 106, 107,
        108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121,
        123, 125, 126, 127, 128, 130, 131, 132, 133, 134, 135, 136, 137, 138,
        139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 152, 153,
        155, 156, 157, 158, 159, 160, 161, 162, 163, 165, 166, 167, 168, 169,
        170, 171, 172, 173, 174, 176, 179, 180, 181, 182, 183, 184, 186, 187,
        188, 189, 193, 194, 195, 196

In [ ]:
# Try explicitly telling the model to ignore padding in attention mask
for batch in packer:
    # Make sure padding tokens have 0 in attention mask
    attention_mask = (batch["input_ids"] != -1).long()
    
    # Convert -1 padding to 0 (or any valid token ID that's safe to use)
    inputs = batch["input_ids"].clone()
    inputs[inputs == -1] = 0
    
    # Fix labels too
    labels = batch["labels"].clone()
    labels[labels == -1] = -100  # -100 is the standard ignore index for CrossEntropyLoss
    
    # Try with the fixed inputs
    outputs = model(
        input_ids=inputs,
        attention_mask=attention_mask,
        labels=labels
    )
    
    break


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


In [ ]:
# Try without using the custom causal mask
for batch in packer:
    inputs = batch["input_ids"].clone()
    inputs[inputs == -1] = 0
    
    # GPT models create their own causal mask internally
    outputs = model(
        input_ids=inputs,
        attention_mask=batch["attention_mask"],
        labels=batch["labels"].clone().masked_fill_(batch["labels"] == -1, -100)
    )
    
    break


In [ ]:
# Inspect the model's embedding parameters
print(f"Model embedding weight shape: {model.transformer.wte.weight.shape}")
print(f"Model embedding weight indices: min={model.transformer.wte.weight.shape[0]-1}, max={0}")


Model embedding weight shape: torch.Size([1000, 768])
Model embedding weight indices: min=999, max=0


In [ ]:
class AutoregressiveSequencePacker(GreedyBestFitSequencePacker):
    def __init__(self, *args, **kwargs):
        kwargs['suppress_masking'] = True
        super().__init__(*args, **kwargs)
    
    def _generate_batches(self):
        """Override to create shifted inputs and labels for autoregressive training"""
        for batch_dict in super()._generate_batches():
            input_ids = batch_dict["input_ids"]
            
            # Create the shifted inputs (tokens 0 to n-1) and labels (tokens 1 to n)
            inputs = input_ids.clone()
            # For labels, we shift by 1 (each position predicts the next token)
            labels = torch.roll(input_ids, shifts=-1, dims=1).clone()
            
            # Handle padding: replace -1 with 0 in inputs
            inputs[inputs == self.pad_token_id] = 0
            
            # Create attention mask (1 for tokens, 0 for padding)
            attention_mask = (input_ids != self.pad_token_id).long()
            
            # For each packed sequence, handle sequence boundaries
            for i, cu_seqlens in enumerate(batch_dict["cu_seqlens"]):
                for j in range(len(cu_seqlens) - 1):
                    start, end = cu_seqlens[j].item(), cu_seqlens[j+1].item()
                    
                    # Set the label at the last position of each sequence to ignore_token_id
                    # We don't want to predict across sequence boundaries
                    labels[i, end-1] = self.ignore_token_id
            
            # Handle padding in labels: replace -1 with ignore_token_id
            labels[labels == self.pad_token_id] = self.ignore_token_id
            
            yield {
                "input_ids": inputs,
                "labels": labels,
                "attention_mask": attention_mask,
                "cu_seqlens": batch_dict["cu_seqlens"],
                "max_seqlen": batch_dict["max_seqlen"]
            }


In [ ]:
# Training loop
model.train()
for epoch in range(3):
    for batch in packer:
        # Move batch to device
        inputs = batch["input_ids"].to(device)
        labels = batch["labels"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        
        # Forward pass - no need for additional conversions
        outputs = model(
            input_ids=inputs,
            attention_mask=attention_mask,
            labels=labels
        )
        
        loss = outputs.loss
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print(f"Loss: {loss.item()}")


IndexError: index out of range in self

In [ ]:
# Debug the exact values that might be causing issues
for batch in packer:
    inputs = batch["input_ids"].clone()
    
    # Replace pad tokens with 0
    inputs[inputs == packer.pad_token_id] = 0
    
    # Check for any values >= vocab_size
    max_val = inputs.max().item()
    print(f"Maximum value in inputs after replacing padding: {max_val}")
    
    # Check if there are any values outside the vocabulary range
    invalid_indices = (inputs >= 1000).nonzero()
    if len(invalid_indices) > 0:
        print(f"Found {len(invalid_indices)} invalid indices")
        print(f"Invalid values: {inputs[invalid_indices[:, 0], invalid_indices[:, 1]]}")
    
    # Try explicitly setting the range of values
    valid_inputs = inputs.clamp(0, 999)
    
    # Check if there are still any issues
    try:
        outputs = model(
            input_ids=valid_inputs.to(device),
            attention_mask=batch["attention_mask"].to(device),
            labels=batch["labels"].clamp(-100, 999).to(device)
        )
        print("Success with clamped values!")
    except Exception as e:
        print(f"Still encountering error: {e}")
    
    break


Maximum value in inputs after replacing padding: 999
Still encountering error: Target -1 is out of bounds.


In [ ]:
# Create a very simple input to test the model directly
test_input = torch.randint(0, 1000, (2, 10)).to(device)
test_labels = torch.randint(0, 1000, (2, 10)).to(device)
test_labels[:, -1] = -100  # Ignore last token
test_mask = torch.ones_like(test_input).to(device)

try:
    test_output = model(
        input_ids=test_input,
        attention_mask=test_mask,
        labels=test_labels
    )
    print(f"Test successful! Loss: {test_output.loss.item()}")
except Exception as e:
    print(f"Test failed: {e}")


Test successful! Loss: 7.109584331512451


In [ ]:
class FixedAutoregressive(GreedyBestFitSequencePacker):
    def __init__(self, *args, vocab_size=1000, **kwargs):
        self.vocab_size = vocab_size
        kwargs['suppress_masking'] = True
        super().__init__(*args, **kwargs)
    
    def _generate_batches(self):
        for batch_dict in super()._generate_batches():
            # Get the input_ids and create a valid version
            input_ids = batch_dict["input_ids"]
            valid_inputs = input_ids.clone()
            valid_inputs[valid_inputs == self.pad_token_id] = 0
            
            # Create attention mask (1 for tokens, 0 for padding)
            attention_mask = (input_ids != self.pad_token_id).long()
            
            # Create shifted labels for autoregressive prediction
            labels = torch.roll(valid_inputs, shifts=-1, dims=1).clone()
            
            # Set padding positions in labels to -100 (not -1)
            padding_mask = (input_ids == self.pad_token_id)
            labels[padding_mask] = -100
            
            # Handle sequence boundaries
            for i, cu_seqlens in enumerate(batch_dict["cu_seqlens"]):
                for j in range(len(cu_seqlens) - 1):
                    end = cu_seqlens[j+1].item()
                    if end > 0 and end < labels.size(1):
                        labels[i, end-1] = -100  # Don't predict across sequence boundaries
            
            # Double-check no -1 values remain in labels
            labels[labels == -1] = -100
            
            yield {
                "input_ids": valid_inputs,
                "labels": labels,
                "attention_mask": attention_mask,
                "cu_seqlens": batch_dict["cu_seqlens"],
                "max_seqlen": batch_dict["max_seqlen"]
            }


In [ ]:
def verify_batch(batch, vocab_size=1000):
    """Check for any values that might cause issues with the model"""
    inputs = batch["input_ids"]
    labels = batch["labels"]
    
    # Check input values
    if inputs.min().item() < 0:
        print(f"Warning: inputs contain values < 0: {inputs.min().item()}")
    if inputs.max().item() >= vocab_size:
        print(f"Warning: inputs contain values >= vocab_size: {inputs.max().item()}")
    
    # Check label values
    valid_label_mask = (labels != -100)
    if valid_label_mask.sum() > 0:
        valid_labels = labels[valid_label_mask]
        if valid_labels.min().item() < 0:
            print(f"Warning: labels contain values < 0 (excluding -100): {valid_labels.min().item()}")
        if valid_labels.max().item() >= vocab_size:
            print(f"Warning: labels contain values >= vocab_size: {valid_labels.max().item()}")
    
    # Check for -1 values
    if (inputs == -1).any():
        print(f"Warning: inputs contain -1 values: {(inputs == -1).sum().item()}")
    if (labels == -1).any():
        print(f"Warning: labels contain -1 values: {(labels == -1).sum().item()}")
    
    return "All checks passed!" if not (
        inputs.min().item() < 0 or 
        inputs.max().item() >= vocab_size or 
        (valid_label_mask.sum() > 0 and (valid_labels.min().item() < 0 or valid_labels.max().item() >= vocab_size)) or
        (inputs == -1).any() or 
        (labels == -1).any()
    ) else "Issues found!"


In [ ]:
verify_batch(batch)

'Issues found!'

In [ ]:
# Fix the current batch
def fix_batch(batch, vocab_size=1000):
    """Fix any problematic values in the batch"""
    # Create fixed copies
    fixed_inputs = batch["input_ids"].clone()
    fixed_labels = batch["labels"].clone()
    
    # Replace -1 with 0 in inputs
    fixed_inputs[fixed_inputs == -1] = 0
    
    # Replace -1 with -100 in labels
    fixed_labels[fixed_labels == -1] = -100
    
    # Ensure all values are in range
    fixed_inputs = fixed_inputs.clamp(0, vocab_size-1)
    
    # For labels, we only clamp the non-ignore values
    valid_mask = (fixed_labels != -100)
    if valid_mask.any():
        fixed_labels[valid_mask] = fixed_labels[valid_mask].clamp(0, vocab_size-1)
    
    # Return the fixed batch
    return {
        "input_ids": fixed_inputs,
        "labels": fixed_labels,
        "attention_mask": batch["attention_mask"],
        "cu_seqlens": batch["cu_seqlens"],
        "max_seqlen": batch["max_seqlen"]
    }

# Try with the fixed batch
fixed = fix_batch(batch)
verify_batch(fixed)

# Try running the model with the fixed batch
try:
    outputs = model(
        input_ids=fixed["input_ids"].to(device),
        attention_mask=fixed["attention_mask"].to(device),
        labels=fixed["labels"].to(device)
    )
    print(f"Success! Loss: {outputs.loss.item()}")
except Exception as e:
    print(f"Error: {e}")


Success! Loss: 7.064399242401123


In [ ]:
class AutoregressiveSequencePacker(GreedyBestFitSequencePacker):
    def __init__(self, *args, vocab_size=1000, **kwargs):
        self.vocab_size = vocab_size
        kwargs['suppress_masking'] = True
        super().__init__(*args, **kwargs)
    
    def _generate_batches(self):
        for batch_dict in super()._generate_batches():
            # Get the input_ids and create a valid version
            input_ids = batch_dict["input_ids"]
            
            # Replace padding with 0 and ensure all values are in range
            valid_inputs = input_ids.clone()
            valid_inputs[valid_inputs == self.pad_token_id] = 0
            valid_inputs = valid_inputs.clamp(0, self.vocab_size-1)
            
            # Create attention mask (1 for tokens, 0 for padding)
            attention_mask = (input_ids != self.pad_token_id).long()
            
            # Create shifted labels for autoregressive prediction
            labels = torch.roll(valid_inputs, shifts=-1, dims=1).clone()
            
            # Mark padding positions with ignore_token_id
            labels[input_ids == self.pad_token_id] = self.ignore_token_id
            
            # Handle sequence boundaries
            for i, cu_seqlens in enumerate(batch_dict["cu_seqlens"]):
                for j in range(len(cu_seqlens) - 1):
                    end = cu_seqlens[j+1].item()
                    if end > 0 and end < labels.size(1):
                        labels[i, end-1] = self.ignore_token_id
            
            # Final safety check - ensure no -1 values remain
            labels[labels == -1] = self.ignore_token_id
            
            yield {
                "input_ids": valid_inputs,
                "labels": labels,
                "attention_mask": attention_mask,
                "cu_seqlens": batch_dict["cu_seqlens"],
                "max_seqlen": batch_dict["max_seqlen"]
            }


In [ ]:
# Training loop
model.train()
for epoch in range(3):
    for batch in packer:
        batch = fix_batch(batch)
        # Move batch to device
        inputs = batch["input_ids"].to(device)
        labels = batch["labels"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        
        # Forward pass - no need for additional conversions
        outputs = model(
            input_ids=inputs,
            attention_mask=attention_mask,
            labels=labels
        )
        
        loss = outputs.loss
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print(f"Loss: {loss.item()}")


Loss: 7.074440002441406
Loss: 7.057838439941406
Loss: 7.064158916473389
Loss: 7.048040390014648
Loss: 7.010573863983154
Loss: 7.013488292694092
Loss: 7.014577388763428
Loss: 6.999983787536621
Loss: 6.993265151977539
Loss: 6.98762321472168


KeyboardInterrupt: 